In [51]:
# Pipeline:

# 1) Design Model (input, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training Loop
#     - forward pass  : compute prediction
#     - backward pass : gradients
#     - update weights

In [52]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [53]:
# prepare data
bc = datasets.load_breast_cancer()

X, y = bc.data, bc.target

n_samples, n_features = X.shape # 569 30

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale 
sc = StandardScaler() # zero mean, unit variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1) # [100, 1]
y_test = y_test.view(y_test.shape[0], 1)

In [54]:
# 1) Model
# f = wx + b, signoid at end
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = LogisticRegression(n_features)

In [55]:
# 2) Loss and Optimizer
learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [56]:
# 3) training loop
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass
    y_pred = model(X_train)
    
    # loss
    loss = criterion(y_pred, y_train) 
    
    # backward pass
    loss.backward()
    
    # update
    optimizer.step()
    
    # zero gradient
    optimizer.zero_grad()
    
    if (epoch+1)%50 == 0:
        [w,b] = model.parameters()
        print('Epoch {}: w = {:.3f}, loss = {:.4f}'.format(epoch, w[0][0].item(), loss))

Epoch 49: w = -0.168, loss = 0.3084
Epoch 99: w = -0.210, loss = 0.2409


In [57]:
# accuracy

with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round() # dont want to track this
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print("Accuracy = {:.4f}".format(acc))

Accuracy = 0.9649
